In [1]:
import os
os.environ["CUDA_DEVICE_OREDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
#from torchsummary import summary
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random
print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

1.8.0 0.2.2
TITAN RTX
11.1


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


In [3]:
# randomness 제어 
# https://hoya012.github.io/blog/reproducible_pytorch/
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [11]:
# dataset load

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset_path = "/media/2/Network/Imagenet_dup/"
# imagenet data load
train_dataset = dataset.ImageFolder(root=dataset_path+"train",
                                       transform=transform)
val_dataset = dataset.ImageFolder(root=dataset_path+"val",
                                       transform=transform)
'''
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=64,
                                        shuffle=False,
                                        num_workers=4)
'''
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=64,
                                        shuffle=True,
                                        num_workers=4) # for using subset
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                        batch_size=64,
                                        shuffle=True, #inital False
                                        num_workers=4)
print(len(train_dataloader),len(val_dataloader))

20019 782


In [5]:
# model 로드 
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)

#if torch.cuda.device_count() > 1:
#    vgg16_bn = nn.DataParallel(vgg16_bn)

In [6]:
# only conv5_1 is trainable
''' hard coding way to freeze weigth
ct = 0
layer_level = 1
back_layer = []
for layer in vgg16_bn.children():
#    print(ct,layer)
    if layer_level is 1:
        for layer_1 in layer:    
            print("layer 1_",ct,layer_1)
            for i,param in enumerate(layer_1.parameters()):
                if ct != 34:
                    param.requires_grad = False
                    #print(ct,"layer false")
                else :
                    if i == 0:
                        param.requires_grad = True
                        print("=====",param.shape)
                        print("%d-%d_layer true"%(ct,i))
                    else :
                        param.requires_grad = False
                        print("=====",param.shape)
                        print("%d-%d_layer false"%(ct,i))
                        
            ct +=1
    elif layer_level is 2:
    #    print("layer 2",layer)  
        for param in layer_1.parameters():
            param.requires_grad = False
            #print(ct,"layer false")
        ct +=1
    elif layer_level is 3:
        for layer_1 in layer: 
    #        print("layer 3",layer_1) 
            for param in layer_1.parameters():
                param.requires_grad = False
                #print(ct,"layer false")
            ct+=1
    layer_level +=1
'''
    

' hard coding way to freeze weigth\nct = 0\nlayer_level = 1\nback_layer = []\nfor layer in vgg16_bn.children():\n#    print(ct,layer)\n    if layer_level is 1:\n        for layer_1 in layer:    \n            print("layer 1_",ct,layer_1)\n            for i,param in enumerate(layer_1.parameters()):\n                if ct != 34:\n                    param.requires_grad = False\n                    #print(ct,"layer false")\n                else :\n                    if i == 0:\n                        param.requires_grad = True\n                        print("=====",param.shape)\n                        print("%d-%d_layer true"%(ct,i))\n                    else :\n                        param.requires_grad = False\n                        print("=====",param.shape)\n                        print("%d-%d_layer false"%(ct,i))\n                        \n            ct +=1\n    elif layer_level is 2:\n    #    print("layer 2",layer)  \n        for param in layer_1.parameters():\n            p

In [7]:

feature_output=[] # extract 10 features from conv5_1 input
def error_injection(name,num_error,start_index):
    def hook(model,input):
        start = start_index
        end = start_index + num_error
        #print("error_injection, input_shape:",input[0][0][start:end].size())
        input[0][0][start:end] = 0
        #print("shape :",input[0][0].size())
        if len(feature_output) < 10:
            feature_output.append(input[0][0].cpu())
    return hook

param_list = []
num_error = 128
error_index = 0
for name,parameter in vgg16_bn.named_parameters():
    if "features.34.weight" in name:
        #print(name,"size:",parameter.size())
        param_list.append(parameter)
for name,layer in vgg16_bn.named_modules():
    #print(name)
    if "features.34" in name  and isinstance(layer, torch.nn.modules.conv.Conv2d):
        print(name,layer) # target layer Conv5_1
        handle = layer.register_forward_pre_hook(error_injection(name,num_error,error_index))


features.34 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [8]:
loss_fn = torch.nn.CrossEntropyLoss(size_average = None).to(device)
#criterion = torch.nn.CrossEntropyLoss(size_average = None).cuda()
optimizer = torch.optim.SGD(param_list,lr=0.005,weight_decay=1e-4)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.95)
max_epochs = 60

In [9]:
# training
def training(model,train_dataloader,optimizer,loss_fn,max_epochs=30,subset=False):
    model.to(device)
    for epoch in range(max_epochs):
        running_loss = 0.0
        print("=====epoch %d start======"%(epoch))
        for i, data in enumerate(train_dataloader):
            inputs,labels = data
            inputs,labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
        
            y_pred = vgg16_bn(inputs)
            # compute loss 
            loss = loss_fn(y_pred,labels)
            #print(epoch,i, loss.item())
            
            if not torch.isfinite(loss):
                print("WARNING: non-finite loss, ending training")
                exit(1)
            else :
                running_loss += loss.item()
            loss.backward()
            optimizer.step()
            if i % 30 == 29: 
                print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/30)) 
                running_loss = 0.0
            if subset is True and i>=400:
                break
training(vgg16_bn,train_dataloader,optimizer,loss_fn,max_epochs,True)               

=====epoch 0 start======
[1,    30] loss: 0.697
[1,    60] loss: 0.757
[1,    90] loss: 0.717
[1,   120] loss: 0.736
[1,   150] loss: 0.705
[1,   180] loss: 0.779
[1,   210] loss: 0.667
[1,   240] loss: 0.731
[1,   270] loss: 0.709
[1,   300] loss: 0.718
[1,   330] loss: 0.640
[1,   360] loss: 0.694
[1,   390] loss: 0.682
=====epoch 1 start======
[2,    30] loss: 0.684
[2,    60] loss: 0.704
[2,    90] loss: 0.721
[2,   120] loss: 0.697
[2,   150] loss: 0.728
[2,   180] loss: 0.662
[2,   210] loss: 0.706
[2,   240] loss: 0.690
[2,   270] loss: 0.732
[2,   300] loss: 0.655
[2,   330] loss: 0.692
[2,   360] loss: 0.731
[2,   390] loss: 0.714
=====epoch 2 start======
[3,    30] loss: 0.706
[3,    60] loss: 0.656
[3,    90] loss: 0.701
[3,   120] loss: 0.788
[3,   150] loss: 0.723
[3,   180] loss: 0.689
[3,   210] loss: 0.678
[3,   240] loss: 0.732
[3,   270] loss: 0.686
[3,   300] loss: 0.714
[3,   330] loss: 0.743
[3,   360] loss: 0.759
[3,   390] loss: 0.703
=====epoch 3 start======
[4,

In [10]:
# evaluation phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)

vgg16_bn.cuda()
vgg16_bn.eval()
total = 0
correct =0
with torch.no_grad():
    print("eval start")
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16_bn(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if(i%50 == 49):
            print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
            #print(".",end="")
    print("")
print("acc of vgg16 on imagenet : %.4f %%" %(100*correct/total))    

eval start
step : 50 / 781
step : 100 / 781
step : 150 / 781
step : 200 / 781
step : 250 / 781
step : 300 / 781
step : 350 / 781
step : 400 / 781
step : 450 / 781
step : 500 / 781
step : 550 / 781
step : 600 / 781
step : 650 / 781
step : 700 / 781
step : 750 / 781

acc of vgg16 on imagenet : 73.5600 %


In [12]:
# evaluation phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)

vgg16_bn.cuda()
vgg16_bn.eval()
total = 0
correct =0
with torch.no_grad():
    print("eval start")
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16_bn(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if(i%50 == 49):
            print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
            #print(".",end="")
    print("")
print("acc of vgg16 not shuffle on imagenet : %.4f %%" %(100*correct/total))    

eval start
step : 50 / 781
step : 100 / 781
step : 150 / 781
step : 200 / 781
step : 250 / 781
step : 300 / 781
step : 350 / 781
step : 400 / 781
step : 450 / 781
step : 500 / 781
step : 550 / 781
step : 600 / 781
step : 650 / 781
step : 700 / 781
step : 750 / 781

acc of vgg16 not shuffle on imagenet : 73.5540 %


In [13]:

vgg16_bn = torchvision.models.vgg16_bn(pretrained=True) # not error inserted
# evaluation phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)

vgg16_bn.cuda()
vgg16_bn.eval()
total = 0
correct =0
with torch.no_grad():
    print("eval start")
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16_bn(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if(i%50 == 49):
            print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
            #print(".",end="")
    print("")
print("acc of vgg16 not shuffle on imagenet : %.4f %%" %(100*correct/total))    

eval start
step : 50 / 781
step : 100 / 781
step : 150 / 781
step : 200 / 781
step : 250 / 781
step : 300 / 781
step : 350 / 781
step : 400 / 781
step : 450 / 781
step : 500 / 781
step : 550 / 781
step : 600 / 781
step : 650 / 781
step : 700 / 781
step : 750 / 781

acc of vgg16 not shuffle on imagenet : 73.3600 %
